In [1]:
# Set up model provider
from IPython.display import display, Markdown, Latex

from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders

model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "gpt-4o"},
))

async def rendered_llm_call(prompt):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response

In [2]:
# Play with these values to experiment
java_code = """
package com.redhat.coolstore.service;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.Transformers;

import javax.inject.Inject;
import javax.jms.*;
import javax.naming.Context;
import javax.naming.InitialContext;
import javax.naming.NamingException;
import javax.rmi.PortableRemoteObject;
import java.util.Hashtable;

public class InventoryNotificationMDB implements MessageListener {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    private final static String JNDI_FACTORY = "weblogic.jndi.WLInitialContextFactory";
    private final static String JMS_FACTORY = "TCF";
    private final static String TOPIC = "topic/orders";
    private TopicConnection tcon;
    private TopicSession tsession;
    private TopicSubscriber tsubscriber;

    public void onMessage(Message rcvMessage) {
        TextMessage msg;
        {
            try {
                System.out.println("received message inventory");
                if (rcvMessage instanceof TextMessage) {
                    msg = (TextMessage) rcvMessage;
                    String orderStr = msg.getBody(String.class);
                    Order order = Transformers.jsonToOrder(orderStr);
                    order.getItemList().forEach(orderItem -> {
                        int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
                        int new_quantity = old_quantity - orderItem.getQuantity();
                        if (new_quantity < LOW_THRESHOLD) {
                            System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
                        } else {
                            orderItem.setQuantity(new_quantity);
                        }
                    });
                }


            } catch (JMSException jmse) {
                System.err.println("An exception occurred: " + jmse.getMessage());
            }
        }
    }

    public void init() throws NamingException, JMSException {
        Context ctx = getInitialContext();
        TopicConnectionFactory tconFactory = (TopicConnectionFactory) PortableRemoteObject.narrow(ctx.lookup(JMS_FACTORY), TopicConnectionFactory.class);
        tcon = tconFactory.createTopicConnection();
        tsession = tcon.createTopicSession(false, Session.AUTO_ACKNOWLEDGE);
        Topic topic = (Topic) PortableRemoteObject.narrow(ctx.lookup(TOPIC), Topic.class);
        tsubscriber = tsession.createSubscriber(topic);
        tsubscriber.setMessageListener(this);
        tcon.start();
    }

    public void close() throws JMSException {
        tsubscriber.close();
        tsession.close();
        tcon.close();
    }

    private static InitialContext getInitialContext() throws NamingException {
        Hashtable<String, String> env = new Hashtable<>();
        env.put(Context.INITIAL_CONTEXT_FACTORY, JNDI_FACTORY);
        env.put(Context.PROVIDER_URL, "t3://localhost:7001");
        env.put("weblogic.jndi.createIntermediateContexts", "true");
        return new InitialContext(env);
    }
}"""

migration_goal = "Migrate the provided Java EE JMS message-driven bean (MDB) to use Quarkus SmallRye reactive messaging."
analyzer_hint = "References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents."

solved_examples = """
Example 1:
- Original: `@MessageDriven(mappedName = "queue/orders")`
- Hint: Replace JMS `@MessageDriven` annotations with `@Incoming` annotations from SmallRye Reactive Messaging.
- Migrated: `@Incoming("queue-orders")`

Example 2:
- Original: Implements `javax.jms.MessageListener` interface.
- Hint: Remove JMS interfaces; methods annotated with `@Incoming` directly receive messages.
- Migrated: Remove implements `MessageListener`; annotate method directly with `@Incoming`
"""
intro = "You are an experienced Java developer, who specializes in migrating code from Java EE to Quarkus."
output_instruction = "Before migrating, reason through the required changes based on the hint provided. Consider changes required in external dependencies (`pom.xml`) and imports. Then, provide the full migrated Java file."

In [3]:
# Scenario 1: Zero-Shot Simple Migration

prompt = f"""
{intro}

Migration Goal:
{migration_goal}

Original Java Code:
```java
{java_code}
```

{output_instruction}
"""

response = await rendered_llm_call(prompt)

Migrating from Java EE JMS to Quarkus with SmallRye Reactive Messaging involves several key changes. Here's a breakdown of the steps and the final migrated code:

### Key Changes:

1. **Dependency Management**: 
   - Add dependencies for SmallRye Reactive Messaging and any connectors you need (e.g., AMQP, Kafka) in your `pom.xml`.

2. **Remove JMS and JNDI**:
   - Eliminate JMS and JNDI-related code. Quarkus handles messaging through annotations and configuration.

3. **Use of Annotations**:
   - Replace the `MessageListener` interface with SmallRye's `@Incoming` annotation to handle incoming messages.

4. **Configuration**:
   - Configure the messaging channel in `application.properties` or `application.yaml`.

5. **Dependency Injection**:
   - Use CDI for injecting services like `CatalogService`.

6. **Logging**:
   - Use a logging framework like SLF4J instead of `System.out.println`.

### Updated `pom.xml` Dependencies:

Ensure you have the following dependencies in your `pom.xml`:

```xml
<dependency>
    <groupId>io.quarkus</groupId>
    <artifactId>quarkus-smallrye-reactive-messaging</artifactId>
</dependency>
<dependency>
    <groupId>io.quarkus</groupId>
    <artifactId>quarkus-smallrye-reactive-messaging-jms</artifactId>
</dependency>
```

### Migrated Java Code:

```java
package com.redhat.coolstore.service;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.Transformers;
import org.eclipse.microprofile.reactive.messaging.Incoming;
import org.jboss.logging.Logger;

import javax.enterprise.context.ApplicationScoped;
import javax.inject.Inject;

@ApplicationScoped
public class InventoryNotificationService {

    private static final Logger LOGGER = Logger.getLogger(InventoryNotificationService.class);
    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    @Incoming("orders")
    public void processOrder(String orderStr) {
        try {
            LOGGER.info("Received message inventory");
            Order order = Transformers.jsonToOrder(orderStr);
            order.getItemList().forEach(orderItem -> {
                int oldQuantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
                int newQuantity = oldQuantity - orderItem.getQuantity();
                if (newQuantity < LOW_THRESHOLD) {
                    LOGGER.warn("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
                } else {
                    orderItem.setQuantity(newQuantity);
                }
            });
        } catch (Exception e) {
            LOGGER.error("An exception occurred: " + e.getMessage(), e);
        }
    }
}
```

### Configuration in `application.properties`:

```properties
mp.messaging.incoming.orders.connector=smallrye-jms
mp.messaging.incoming.orders.destination=topic/orders
mp.messaging.incoming.orders.host=localhost
mp.messaging.incoming.orders.port=7001
```

### Explanation:

- **Annotations**: The `@Incoming("orders")` annotation replaces the JMS listener setup. It listens to the `orders` channel.
- **Logging**: Replaced `System.out.println` with `Logger` for better logging practices.
- **Configuration**: Messaging configurations are moved to `application.properties`, simplifying the code and making it more flexible.
- **Dependency Injection**: `CatalogService` is injected using CDI, similar to the original code.

This migration leverages Quarkus's reactive messaging capabilities, simplifying the code and improving scalability and maintainability.

In [4]:
# Scenario 2: Migration with Analyzer Hint

prompt = f"""
{intro}

Migration Goal:
{migration_goal}

Analyzer Hint:
{analyzer_hint}

Original Java Code:
```java
{java_code}
```

{output_instruction}
"""

response = await rendered_llm_call(prompt)

To migrate the Java EE JMS message-driven bean (MDB) to use Quarkus SmallRye reactive messaging, we need to replace the JMS-specific elements with SmallRye's reactive messaging constructs. This involves using annotations and classes provided by SmallRye to handle messaging in a reactive way.

### Key Changes:

1. **Remove JMS-specific Classes and Interfaces**: We will remove `MessageListener`, `TopicConnection`, `TopicSession`, `TopicSubscriber`, and other JMS-specific classes.

2. **Use SmallRye Reactive Messaging Annotations**: We will use `@Incoming` to listen to messages from a channel.

3. **Dependency Management**: Ensure that the `pom.xml` includes dependencies for SmallRye Reactive Messaging.

4. **Simplify Context and Connection Management**: SmallRye handles connections and context internally, so we don't need to manually manage them.

5. **Message Handling**: We will directly handle the message payload using SmallRye's reactive messaging capabilities.

### Updated `pom.xml` Dependencies:

Ensure you have the following dependencies in your `pom.xml`:

```xml
<dependency>
    <groupId>io.quarkus</groupId>
    <artifactId>quarkus-smallrye-reactive-messaging</artifactId>
</dependency>
<dependency>
    <groupId>io.quarkus</groupId>
    <artifactId>quarkus-smallrye-reactive-messaging-jms</artifactId>
</dependency>
```

### Migrated Java Code:

```java
package com.redhat.coolstore.service;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.Transformers;
import io.smallrye.reactive.messaging.annotations.Blocking;
import org.eclipse.microprofile.reactive.messaging.Incoming;

import javax.enterprise.context.ApplicationScoped;
import javax.inject.Inject;

@ApplicationScoped
public class InventoryNotificationService {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    @Incoming("orders")
    @Blocking
    public void processOrder(String orderStr) {
        try {
            System.out.println("Received message inventory");
            Order order = Transformers.jsonToOrder(orderStr);
            order.getItemList().forEach(orderItem -> {
                int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
                int new_quantity = old_quantity - orderItem.getQuantity();
                if (new_quantity < LOW_THRESHOLD) {
                    System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
                } else {
                    orderItem.setQuantity(new_quantity);
                }
            });
        } catch (Exception e) {
            System.err.println("An exception occurred: " + e.getMessage());
        }
    }
}
```

### Explanation:

- **@ApplicationScoped**: This annotation is used to define a CDI bean with application scope.
- **@Incoming("orders")**: This annotation listens to messages from the "orders" channel. You need to configure this channel in your `application.properties` or `application.yaml`.
- **@Blocking**: This annotation ensures that the method runs on a worker thread, which is useful for blocking operations.
- **Simplified Message Handling**: The method `processOrder` directly processes the incoming message payload as a `String`.

### Configuration:

Ensure that your `application.properties` or `application.yaml` is configured to define the "orders" channel. For example:

```properties
mp.messaging.incoming.orders.connector=smallrye-jms
mp.messaging.incoming.orders.destination=topic/orders
mp.messaging.incoming.orders.durable=false
```

This configuration will depend on your specific setup and JMS broker. Adjust the properties accordingly.

In [5]:
# Scenario 3: Migration with Examples and Hint

prompt = f"""
{intro}

Consider these migration examples:
{solved_examples}

Now apply similar logic to the task below:

Migration Goal:
{migration_goal}

Analyzer Hint:
{analyzer_hint}

Original Java Code:
```java
{java_code}
```

{output_instruction}
"""

response = await rendered_llm_call(prompt)

To migrate the provided Java EE JMS message-driven bean (MDB) to use Quarkus SmallRye Reactive Messaging, we need to replace the JMS-specific elements with their SmallRye equivalents. This involves removing the JMS setup code and using the `@Incoming` annotation to handle incoming messages. Additionally, we need to adjust the imports and remove unnecessary JMS-related code.

Here's the full migrated Java file:

```java
package com.redhat.coolstore.service;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.Transformers;
import io.smallrye.reactive.messaging.annotations.Blocking;
import org.eclipse.microprofile.reactive.messaging.Incoming;

import javax.inject.Inject;

public class InventoryNotificationService {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    @Incoming("orders")
    @Blocking
    public void processOrder(String orderStr) {
        System.out.println("received message inventory");
        Order order = Transformers.jsonToOrder(orderStr);
        order.getItemList().forEach(orderItem -> {
            int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
            int new_quantity = old_quantity - orderItem.getQuantity();
            if (new_quantity < LOW_THRESHOLD) {
                System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
            } else {
                orderItem.setQuantity(new_quantity);
            }
        });
    }
}
```

### Key Changes:

1. **Remove JMS Setup Code**: The JMS connection setup, session creation, and message listener registration are removed. These are not needed with SmallRye Reactive Messaging.

2. **Use `@Incoming` Annotation**: The `@Incoming("orders")` annotation is used to specify that the method should be invoked when a message is received on the "orders" channel. This replaces the `MessageListener` interface and the `onMessage` method.

3. **Method Signature Change**: The method now directly receives the message payload as a `String`, which is the body of the `TextMessage`.

4. **Remove JMS Exception Handling**: Since we're no longer dealing with JMS directly, the `JMSException` handling is removed.

5. **Blocking Annotation**: The `@Blocking` annotation is used to ensure that the processing of the message is done on a worker thread, which is important if the processing involves blocking operations like database access.

6. **Imports Update**: The imports are updated to include SmallRye and MicroProfile Reactive Messaging annotations and remove JMS-related imports.

### Additional Considerations:

- **Dependencies**: Ensure that your `pom.xml` includes the necessary dependencies for SmallRye Reactive Messaging and any connectors you might need (e.g., Kafka, AMQP).

- **Configuration**: Configure the messaging channels in your `application.properties` or `application.yml` file to specify how messages are received on the "orders" channel. This might involve setting up a connector to a messaging broker.

By following these steps, the code is now adapted to use Quarkus SmallRye Reactive Messaging, which is more suitable for modern cloud-native applications.